In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [2]:
import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
from numpy.random import seed
from sklearn.utils import class_weight

from inception_model_generator import MRNet_inc_model

import tensorflow as tf


Using TensorFlow backend.


In [0]:
label_type = 'abnormal'
exam_view = 'axial'
model, cbs = MRNet_inc_model(2,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
class_count_train = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
class_count_test = {0: neg, 1: pos}

batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_train , batch_size=1,model="inception",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=1)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_test, batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=1)

# model.load_weights('/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/abnormal/axial/weights.05.hdf5')
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=20, initial_epoch = 0,verbose=True, callbacks=[cbs],shuffle=False, use_multiprocessing=True, workers=6)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_1 (MR (None, 1)                 23066273  
Total params: 23,066,273
Trainable params: 23,066,273
Non-trainable params: 0
_________________________________________________________________
Initializing Data Generator:
factor :  4.242268041237113
model:  inception
data type:  train
Combination:  abnormal  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1646
input size:  (299, 299)
Initializing Data Generator:
factor :  3.9130434782608696
model:  inception
data type:  test
Combination:  abnormal  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  180
input size:  (299, 299)
Epoch 1/20
   3/1646 [..............................] - ETA: 8:17:43 - loss: 5.0076 - tp: 0.0000e+00 - fp: 2.0000 - tn

In [0]:
b_size = 1

data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)


In [0]:

valid_gen = MRNet_data_generator(data_path, p, l, batch_size=b_size, scale_to=(299,299),train=False, shuffle=False,model="inc",aug_size=0)
print(valid_gen.__len__());

In [0]:
model, cp_callback = MRNet_inc_model(["abnormal", "axial"])
os.chdir('/content/gdrive/My Drive/Colab Notebooks/MRNet')

model.load_weights('training_inception/abnormal/axial/cp.ckpt');


In [0]:
d = model.evaluate(x=valid_gen,verbose=1,use_multiprocessing=True,workers=6);
print(d)


In [0]:
model.metrics_names

In [0]:

d = model.predict(x=valid_gen,verbose=1,use_multiprocessing=True,workers=6)
print(d)

In [0]:

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 